In [1]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



#Initial Load boston_table_id = "opendatadbt.311.boston311"

boston311 = requests.get("https://data.boston.gov/api/3/action/datastore_search_sql?sql=SELECT%20*%20from%20%22f53ebccd-bc61-49f9-83db-625f209c95f5%22%20WHERE%20%22open_dt%22%20%3E=%20%272021-03-22%27" ) 
boston311json = boston311.json() 
boston_df = pd.json_normalize(boston311json['result']['records']) 
boston_df



In [3]:
#Incremental load
boston_table_id = "opendatadbt.311.boston311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")

query_string = """
SELECT 
max(open_dt), max(closed_dt)
FROM `opendatadbt.311.boston311`
limit 10
"""

bostonmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
bostonmaxopendatedate = str(bostonmaxopendatedataframe['f0_'][0])

bostonmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
bostonmaxclosedate = str(bostonmaxclosedatedataframe['f1_'][0])


In [4]:
boston311 = requests.get("https://data.boston.gov/api/3/action/datastore_search_sql?sql=SELECT%20*%20from%20%22f53ebccd-bc61-49f9-83db-625f209c95f5%22%20WHERE%20%22open_dt%22>%20%27" + bostonmaxopendatedate +"%27 OR closed_dt between '" +bostonmaxclosedate+ "' AND '" + current_datetime + "'") 
boston311json = boston311.json()
cols = ['case_enquiry_id','open_dt','closed_dt','department','subject','case_title','location_zipcode','location','neighborhood','case_status','latitude','longitude']
boston_df = pd.json_normalize(boston311json['result']['records'])
boston_df = pd.DataFrame(boston_df, columns = cols)
boston_df

,case_enquiry_id,open_dt,closed_dt,department,subject,case_title,location_zipcode,location,neighborhood,case_status,latitude,longitude
0,101003690728,2021-03-27 13:19:00,None,BWSC,Boston Water & Sewer Commission,Catchbasin,02119,3 Dunford St Roxbury MA 02119,Roxbury,Open,42.3175,-71.0945
1,101003591838,2021-01-17 14:51:01,2021-03-27 11:27:26,PROP,Property Management,Graffiti Removal,None,INTERSECTION of Ralphs Way & Brookline Ave Bo...,Fenway / Kenmore / Audubon Circle / Longwood,Closed,42.3594,-71.0587
2,101003592135,2021-01-18 08:24:22,2021-03-27 08:24:13,PROP,Property Management,Graffiti Removal,02118,105 W Springfield St Roxbury MA 02118,South End,Closed,42.3381,-71.0775
3,101003630571,2021-02-16 20:14:00,2021-03-27 10:25:51,PWDx,Public Works Department,Street Light Outages//Issued to Cont. No power...,02127,637 E First St South Boston MA 02127,South Boston / South Boston Waterfront,Closed,42.3381,-71.034
4,101003641725,2021-02-17 21:38:00,2021-03-27 10:21:29,PWDx,Public Works Department,Street Light Outages//CONTRACTOR//,None,INTERSECTION of Summer St & E First St South ...,South Boston / South Boston Waterfront,Closed,42.3594,-71.0587
...,...,...,...,...,...,...,...,...,...,...,...,...
565,101003691066,2021-03-27 22:46:53,None,BTDT,Transportation - Traffic Division,Parking Enforcement,None,INTERSECTION of Centre Ct & Centre St Dorches...,Dorchester,Open,42.3594,-71.0587
566,101003691070,2021-03-27 22:58:04,None,PARK,Parks & Recreation Department,Tree Maintenance Requests,02135,1749 Commonwealth Ave Brighton MA 02135,Allston / Brighton,Open,42.3417,-71.1498
567,101003691073,2021-03-27 23:12:12,None,PWDx,Public Works Department,Street Light Outages,02135,1970 Commonwealth Ave Brighton MA 02135,Allston / Brighton,Open,42.339,-71.1568
568,101003691082,2021-03-27 23:48:56,None,PWDx,Public Works Department,Empty Litter Basket,None,INTERSECTION of Sutherland Rd & Commonwealth A...,Allston / Brighton,Open,42.3594,-71.0587


In [5]:

boston_table_id = "opendatadbt.311.boston311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('boston311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    boston_df, table_ref, job_config=job_config
).result()  # Make an API request.
